In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import pickle
import pandas as pd
import numpy as np
import bz2
import re
import matplotlib.pyplot as plt
import sklearn
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
import seaborn as sns
sns.set(rc={'figure.figsize':(11,5)})

In [ ]:
real_news = pd.read_csv('../input/fake-and-real-news-dataset/True.csv')
real_news['label'] = 1
real_news.info()

In [ ]:
real_news.head()

In [ ]:
fake_news = pd.read_csv('../input/fake-and-real-news-dataset//Fake.csv')
fake_news['label'] = 0
fake_news.info()

In [ ]:
news = pd.concat([real_news, fake_news], ignore_index = True, sort = False)
news.head()

In [ ]:
pd.set_option('max_colwidth', True)
news.head(1)

In [ ]:
sns.countplot(x= news.label)
print(news["label"].value_counts())

In [ ]:
sns.countplot(x="subject",  hue='label', data=news, palette="Set1")
plt.title("Distribution of The Subject According to Real and Fake Data")

In [ ]:
news['date'] = news['date'].str.replace('Dec ', 'December ')
news['date'] = news['date'].str.replace('Nov ', 'November ')
news['date'] = news['date'].str.replace('Oct ', 'October ')
news['date'] = news['date'].str.replace('Sep ', 'September ')
news['date'] = news['date'].str.replace('Aug ', 'August ')
news['date'] = news['date'].str.replace('Jul ', 'July ')
news['date'] = news['date'].str.replace('Jun ', 'June ')
news['date'] = news['date'].str.replace('Apr ', 'April ')
news['date'] = news['date'].str.replace('Mar ', 'March ')
news['date'] = news['date'].str.replace('Feb ', 'February ')
news['date'] = news['date'].str.replace('Jan ', 'January ')

In [ ]:
news['date'] = news['date'].str.replace(' ', '')
news['date'] = pd.to_datetime(news['date'], format='%B%d,%Y', errors = 'coerce')
news['date'] = news['date'].astype('datetime64')
news.info()

In [ ]:
yearly_news_fall = news[['date', 'label']]
yearly_news_fall = news.groupby(['date'])['label'].count()
yearly_news_fall = pd.DataFrame(yearly_news_fall)
yearly_news_fall

In [ ]:
news['year'] = news['date'].dt.year
news['month'] = news['date'].dt.month

news.head(1)

In [ ]:
news_replica = news.copy()

In [ ]:
news_replica['news_stream'] = news_replica['subject'] + " " + news_replica['title'] + " " + news_replica['text']
news_replica.drop(['title', 'text', 'subject','date','year','month'], axis=1, inplace=True)

In [ ]:
news_replica.info()

In [ ]:
from nltk.corpus import stopwords
# nltk.download('stopwords')

In [ ]:
from wordcloud import WordCloud,STOPWORDS
plt.figure(figsize = (15,15))
wc = WordCloud(max_words = 500 , width = 1000 , height = 500 ,background_color = 'white', stopwords = STOPWORDS).generate(" ".join(news_replica[news_replica.label == 1].news_stream))
plt.imshow(wc , interpolation = 'bilinear')

In [ ]:
from wordcloud import WordCloud,STOPWORDS
plt.figure(figsize = (15,15))
wc = WordCloud(max_words = 500 , width = 1000 , height = 500, background_color = 'white' , stopwords = STOPWORDS).generate(" ".join(news_replica[news_replica.label == 0].news_stream))
plt.imshow(wc , interpolation = 'bilinear')

In [ ]:
# lemma = nltk.WordNetLemmatizer()
# corpus_1 = []

# for i in range(0, len(news_replica.head(10000))):
#     print(i)
#     review = re.sub('[^a-zA-Z]', ' ', news_replica['news_stream'][i])
#     review = review.lower()
#     review = nltk.word_tokenize(review)
    
#     review = [lemma.lemmatize(word) for word in review if not word in stopwords.words('english')]
#     review = ' '.join(review)
#     corpus_1.append(review)

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strata = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
with tpu_strata.scope():
    lemma = nltk.WordNetLemmatizer()
    corpus = []

    for i in range(0, len(news_replica)):
        review = re.sub('[^a-zA-Z]', ' ', news_replica['news_stream'][i])
        review = review.lower()
        review = nltk.word_tokenize(review)

        review = [lemma.lemmatize(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)

In [ ]:
print(len(corpus))
print(corpus[1])

In [ ]:


pickle_out = open("corpus.pickle","wb")
pickle.dump(corpus, pickle_out)
pickle_out.close()

In [ ]:
with open('../input/corpus/corpus.pickle', 'rb') as file:
    corpus = pickle.load(file)

In [ ]:
labels = np.array(news_replica.label)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

In [ ]:
word_index = tokenizer.word_index
v_size = len(word_index)

In [ ]:
print(len(word_index))

In [ ]:
max_len = 300
max_features = 10000
padding_type = 'pre'
embed_size = 100

sequences = tokenizer.texts_to_sequences(corpus)
padded = pad_sequences(sequences, maxlen = max_len, padding = padding_type)

In [ ]:

test_portion = 0.3
training_size = 35000

split = int(test_portion * training_size)

test_seq = padded[0:split]
train_seq = padded[split: training_size]
test_labels = labels[0:split]
train_labels = labels[split:training_size]

In [ ]:
test_seq = np.array(test_seq)
train_seq = np.array(train_seq)

In [ ]:
print(test_seq.shape)
print(train_seq.shape)
print(test_labels.shape)
print(train_labels.shape)


In [ ]:
print(v_size)
print(word_index['news'])

In [ ]:

tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strata = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
BATCH_SIZE = 16* tpu_strata.num_replicas_in_sync
with tpu_strata.scope():
    model_lstm = tf.keras.Sequential([
        tf.keras.layers.Embedding(v_size+1, embed_size, input_length = max_len, trainable = False),
        tf.keras.layers.LSTM(128, recurrent_dropout = 0.25, dropout = 0.25, return_sequences = True),
        tf.keras.layers.LSTM(128),
        tf.keras.layers.Dense(64, activation = 'relu'),
        tf.keras.layers.Dense(32, activation = 'relu'),
        tf.keras.layers.Dense(1, activation = 'sigmoid')
    ])

    model_lstm.compile('adam', 'binary_crossentropy', metrics = ['accuracy'], steps_per_execution=16)
    model_lstm.summary()

history_lstm = model_lstm.fit(train_seq, train_labels, batch_size = BATCH_SIZE, epochs = 20, validation_data = (test_seq, test_labels), verbose = 2)

In [ ]:
import matplotlib.image  as mpimg

def plot_graphs(history):
    #-----------------------------------------------------------
    # Retrieve a list of list results on training and test data
    # sets for each training epoch
    #-----------------------------------------------------------
    acc=history.history['accuracy']
    val_acc=history.history['val_accuracy']
    loss=history.history['loss']
    val_loss=history.history['val_loss']

    epochs=range(len(acc)) # Get number of epochs

    #------------------------------------------------
    # Plot training and validation accuracy per epoch
    #------------------------------------------------
    plt.plot(epochs, acc, 'r')
    plt.plot(epochs, val_acc, 'b')
    plt.title('Training and validation accuracy')
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend(["Accuracy", "Validation Accuracy"])

    plt.figure()

    #------------------------------------------------
    # Plot training and validation loss per epoch
    #------------------------------------------------
    plt.plot(epochs, loss, 'r')
    plt.plot(epochs, val_loss, 'b')
    plt.title('Training and validation loss')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend(["Loss", "Validation Loss"])

    plt.figure()

plot_graphs(history_lstm)

In [ ]:
# fig, ax = plt.subplots(figsize=(14,6))
# ax.plot(history_lstm.history['loss'], 'b' ,label = 'train loss', linewidth=2)
# ax.plot(history_lstm.history['val_loss'], 'r', label ='Validation loss', linewidth=2)
# ax.set_title('model loss')
# ax.set_ylabel('mse')
# ax.set_xlabel('epoch')
# ax.legend()
# plt.show()

In [ ]:
print("Accuracy of the model on Training Data is - " , model_lstm.evaluate(train_seq, train_labels)[1]*100 , "%")
print("Accuracy of the model on Testing Data is - " , model_lstm.evaluate(test_seq, test_labels)[1]*100 , "%")

-----
## Different approach
### going pass the baseline model using tensorflow's **_TextVectorization()_** utility


In [ ]:
texts = np.array(corpus)


In [ ]:
labels = np.array(news_replica.label)

In [ ]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(texts, labels, test_size= 0.3, random_state= 42)

In [ ]:
train_sentences.shape, val_sentences.shape, train_labels.shape, val_labels.shape

In [ ]:
max_len = round(sum([len(i.split()) for i in train_sentences]) / len(train_sentences))
max_len

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


max_vocab_len = 10000

text_vector = TextVectorization(max_tokens = max_vocab_len, 
                                standardize = "lower_and_strip_punctuation",
                                split = 'whitespace',
                                ngrams = None,
                                output_mode = 'int',
                                output_sequence_length = max_len,
                                pad_to_max_tokens = True,
                                vocabulary = None)



In [ ]:
#fit the text vector to the training text
text_vector.adapt(train_sentences)

In [ ]:
import random
random_sentence = random.choice(train_sentences)
print(f"original text : \n {random_sentence} \n\n Vectorized format : \n{text_vector([random_sentence])} \n\n shape : {text_vector([random_sentence]).shape}")

In [ ]:
print(f" 5 most common words : {text_vector.get_vocabulary()[:5]} \n \n 5 least common words : {text_vector.get_vocabulary()[-5:]}")

In [ ]:
#Embedding layer

embedding = layers.Embedding(input_dim = max_vocab_len, 
                      output_dim = 128,
                      embeddings_initializer= 'uniform',
                      input_length = max_len)


In [ ]:
print(f"original text : \n {random_sentence} \n\n Embedded format : \n{embedding(text_vector([random_sentence]))} \n\n shape : {embedding(text_vector([random_sentence])).shape}")

In [ ]:

inputs = layers.Input(shape = (1,), dtype = 'string')
x = text_vector(inputs)
x = embedding(x)
x = layers.LSTM(64, return_sequences = True, recurrent_dropout = 0.25, dropout = 0.25)(x)
x = layers.LSTM(64)(x)
x = layers.Dense(32, activation = 'relu')(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
lstm = tf.keras.Model(inputs, outputs, name = 'LSTM_MODEL')
lstm.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = 'accuracy')

lstm_history = lstm.fit(train_sentences, train_labels, epochs = 5,  batch_size = 32, validation_data = (val_sentences, val_labels))







In [ ]:
lstm_pred = lstm.predict(val_sentences)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def model_scores(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [ ]:
lstm_score = model_scores(val_labels,  tf.squeeze(tf.round(lstm_pred)))
lstm_score

In [ ]:
plot_graphs(lstm_history)

In [ ]:
print("Accuracy of the model on Training Data is - " , lstm.evaluate(train_sentences, train_labels)[1]*100 , "%")
print("Accuracy of the model on Testing Data is - " , lstm.evaluate(val_sentences, val_labels)[1]*100 , "%")